In [2]:
import json
from math import sin

from bokeh.io import output_file, show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Select
from bokeh.plotting import figure


##output_file("foo.html")

xl=list(range(100))
xs=list(range(500))
xl, yl = list(range(100)), [sin(x) for x in xl]
xs, ys = list(range(500)), [sin(x) for x in xs]
source = ColumnDataSource(data=dict(x=xl, y=yl))

plot = figure()
plot.line('x', 'y', source=source)

select = Select(title="Event:", value="long", options=["long", "short"])

callback = CustomJS(args=dict(source=source, select=select), code="""
    data = %s;
    if (select.value == "long") {
        source.data['x'] = data.xl
        source.data['y'] = data.yl
    } else {
        source.data['x'] = data.xs
        source.data['y'] = data.ys
    }
    source.change.emit()
""" % json.dumps(dict(xl=xl, yl=yl, xs=xs, ys=ys)))

select.callback = callback

show(column(select, plot))

In [3]:
from bokeh.io import show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.models.widgets import RadioButtonGroup
from bokeh.layouts import column, widgetbox
from bokeh.models.callbacks import CustomJS

output_file("radio.html")

dataset = {'x':[0,1,2],'y':[0,1,2],'x_filter':[0,1,2]}

source = ColumnDataSource(data=dataset)
p = figure(plot_width=600, plot_height=600, 
			x_range=(-0.1, 2.1), y_range=(-0.1,2.1))
p.scatter('x', 'y', source=source, 
		  size=15, alpha=0.8, line_color=None)

# add callback to control 
callback = CustomJS(args=dict(p=p, source=source), code="""

			var radio_value = cb_obj.active;
			var data = source.data;            
			x = data['x']
			x_filter = data['x_filter']
			y = data['y']

			for (i = 0; i < x.length; i++) {
				if(x_filter[i] == radio_value) {
					console.log('radio_value=',radio_value)
					x[i] = x_filter[i];
				} else {
					x[i] = undefined;
				}
			}
		source.change.emit();
		""")

# option
option = RadioButtonGroup(labels=["One", "Two", "Three"],
						  active=0, callback=callback)
show(column(widgetbox(option),p))

'''
For flask
script, div = components(column(widgetbox(option),p))
res_json = jsonify(dict(script=script, div=div))
return res_json, 200, {'Content-Type': 'application/json'}
'''

"\nFor flask\nscript, div = components(column(widgetbox(option),p))\nres_json = jsonify(dict(script=script, div=div))\nreturn res_json, 200, {'Content-Type': 'application/json'}\n"

In [4]:
from bokeh.plotting import figure, output_file
from bokeh.models import CustomJS, ColumnDataSource, Select
from bokeh.layouts import gridplot
from bokeh.resources import CDN
from bokeh.embed import file_html

from random import random

key_list = list('ABCDEFGHIJKLMNOP')

DATA1 = {key:[random() for i in range(10)] for key in key_list}
DATA2 = {key:[random() for i in range(10)] for key in key_list}
DATA1['xaxis'] = range(10)
DATA2['xaxis'] = range(10)

source1 = ColumnDataSource(data=DATA1)
source2 = ColumnDataSource(data=DATA2)

fill_source = ColumnDataSource(data=DATA1)

fig = figure()

for key in key_list:
    fig.line(x='xaxis',y=key,source=fill_source)

select = Select(options=['source1','source2'],value='source1')

codes = """
var f = cb_obj.value;
var sdata = source.data;
var data1 = source1.data;
var data2 = source2.data;

console.log(data2);

for (key in data1) {console.log(key);}

if (f == "source1") {
for (key in data1) {
    sdata[key] = [];
    for (i=0;i<data1[key].length;i++){
    sdata[key].push(data1[key][i]);
    }
}
} else {
for (key in data2) {
    sdata[key] = [];
    for (i=0;i<data2[key].length;i++){
    sdata[key].push(data2[key][i]);
    }
}
};
source.change.emit()
//source.trigger("change");
"""
select.callback = CustomJS(args=dict(source=fill_source,source1=source1,source2=source2),code=codes)

grid = gridplot([[select],[fig]])

outfile=open('select.html','w')
outfile.write(file_html(grid,CDN,'select'))
outfile.close()

In [ ]:
#DataPicker에 의해 x_range를 바꿈
import numpy as np
from bokeh.models import CustomJS, DatePicker, TextInput, ColumnDataSource
from bokeh.layouts import row, layout
from bokeh.plotting import output_file, show, figure
from datetime import datetime, timedelta

output_file("DatePicker.html")

# Figure
f=figure(x_axis_type='datetime')

# Datetime
date_time = [datetime(2017,1,1) + timedelta(days=x) for x in range(0,365)]

# ColumnDataSource
source = ColumnDataSource(dict(datetime=date_time, parameter=np.random.rand(365)))

# Line for plot
f.line(x='datetime',y='parameter',line_color='black',source=source)

# Callback
callbackPlot= CustomJS(args=dict(f=f), code="""
    var a = cb_obj.value;
    f.x_range.start = Date.parse(a);
    console.log(Date.parse(a));
""")

# Datepicker
DatepickerPlot = DatePicker(
    title="Start:", name="StartDate",  value=date_time[0])
DatepickerPlot.js_on_change('value', callbackPlot)

# Show 
show(layout(row(DatepickerPlot, f)))

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models.widgets import Tabs, Panel
output_file("tabPanel.html")

panels = []

for axis_type in ["linear", "log"]:
    fig = figure(x_axis_type=axis_type, y_axis_type=axis_type)
    fig.scatter(x=[1,10,100,1000], y=[1,10,100,1000])

    panel = Panel(child=fig, title=axis_type)
    panels.append(panel)

tabs = Tabs(tabs=panels)

show(tabs)

In [ ]:
#not working
import numpy as np

from bokeh.models import ColumnDataSource
from bokeh.plotting import Figure

from bokeh.models.widgets import Select,TextInput
from bokeh.layouts import row, column
import bokeh.io 
from bokeh.models import CustomJS

output_file("changeInput.html")

N = 200

# Define the data to be used
x = np.linspace(0,4.*np.pi,N)
y = 3*np.cos(2*np.pi*x + np.pi*0.2)
z = 0.5*np.sin(2*np.pi*0.8*x + np.pi*0.4)

source = ColumnDataSource(data={'x':x,'y':y, 'X': x, 'cos':y,'sin':z})


code="""
        var data = source.data;
        var r = cb_obj.value;
        var {var} = cb_obj.value;
        //window.alert( "{var} " + cb_obj.value + {var}  );
        for (i = 0; i < r.length; i++) {{
            {var}[i] = r[i] ;
            data['{var}'][i] = r[i];
        }}
        source.change.emit();
    """

callbackx = CustomJS(args=dict(source=source), code=code.format(var="x"))
callbacky = CustomJS(args=dict(source=source), code=code.format(var="y"))

# create a new plot 
plot = Figure(title=None)

# Make a line and connect to data source
plot.line(x="x", y="y", line_color="#F46D43", line_width=6, line_alpha=0.6, source=source)


# Add list boxes for selecting which columns to plot on the x and y axis
yaxis_select = Select(title="Y axis:", value="cos",
                           options=['X','cos','sin'], callback=callbacky)


xaxis_select = Select(title="X axis:", value="x",
                           options=['X','cos','sin'], callback=callbackx)


# Text input as a title
text = TextInput(title="title", value='my sine wave plotter')

# Layout widgets next to the plot                     
controls = column(text,yaxis_select,xaxis_select)

layout = row(controls,plot,width=800)

bokeh.io.show(layout)

In [ ]:
import pandas as pd
import numpy as np

from bokeh.io import show
from bokeh.layouts import widgetbox,row
from bokeh.models import ColumnDataSource, CustomJS

output_file("dropdown.html")

df = pd.DataFrame()
df['x'] = np.random.randint(1,1000,1000)
df['y'] = np.random.randint(1,1000,1000)
df['val1'] = np.random.randint(1,1000,1000)
df['val2'] = np.random.randint(1,1000,1000)
df['val3'] = np.random.randint(1,1000,1000)

from bokeh.plotting import figure
from bokeh.models import LinearColorMapper
from bokeh.palettes import RdYlBu11 as palette

p = figure(x_range=(0,1000),y_range=(0,1000))
source = ColumnDataSource(df)
source_orig = ColumnDataSource(df)
color_mapper = LinearColorMapper(palette=palette)
p.rect('x', 'y', source=source,width=4,height=4,
          color={'field': 'val1', 'transform':color_mapper})

from bokeh.models.widgets import Select
select = Select(title="Option:", value="val1", options=["val1","val2","val3"])

callback = CustomJS(args={'source':source},code="""
        // print the selectd value of the select widget - 
        // this is printed in the browser console.
        // cb_obj is the callback object, in this case the select 
        // widget. cb_obj.value is the selected value.
        console.log(' changed selected option', cb_obj.value);

        // create a new variable for the data of the column data source
        // this is linked to the plot
        var data = source.data;

        // allocate the selected column to the field for the y values
        data['y'] = data[cb_obj.value];

        // register the change - this is required to process the change in 
        // the y values
        source.change.emit();
""")

# Add the callback to the select widget. 
# This executes each time the selected option changes
select.callback = callback
show(row(p,select))

In [ ]:
import pandas as pd

from bokeh.palettes import Spectral4
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.sampledata.stocks import AAPL, IBM, MSFT, GOOG

p = figure(plot_width=800, plot_height=250, x_axis_type="datetime")
p.title.text = 'Click on legend entries to hide the corresponding lines'

for data, name, color in zip([AAPL, IBM, MSFT, GOOG], ["AAPL", "IBM", "MSFT", "GOOG"], Spectral4):
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'])
    p.line(df['date'], df['close'], line_width=2, color=color, alpha=0.8, legend=name)

p.legend.location = "top_left"
p.legend.click_policy="hide"

output_notebook()

show(p)